# Arista vEOSを使ったJupyterの実験
- 参考サイト
    - https://eos.arista.com/ja/eapi-j-01/
        - コード例がPython2なので注意
    - https://eos.arista.com/script-example-automating-vxlan-deployments-with-eapi/

## 事前設定
- Switch1
```
enable
configure
hostname vEOS1
username apitest priv 15 secret password
int ma1
ip address 192.168.0.1/24
no shut
management api http
protocol http
no shut
end
copy run start
```

- Switch2
```
enable
configure
hostname vEOS2
username apitest priv 15 secret password
int ma1
ip address 192.168.0.2/24
no shut
management api http
protocol http
no shut
end
copy run start
```

- vEOSノードと疎通できているかを確認する

In [ ]:
!ping -c 3 192.168.0.1
!ping -c 3 192.168.0.2

In [ ]:
vEOS1 = '192.168.0.1'
vEOS2 = '192.168.0.2'

import os
response = os.system("ping -c 1 " + vEOS1)

#and then check the response...
if response == 0:
  print(vEOS1, 'is up!')
else:
  print(vEOS1, 'is down!')

In [ ]:
response = os.system("ping -c 1 " + vEOS2)

#and then check the response...
if response == 0:
  print(vEOS2, 'is up!')
else:
  print(vEOS2, 'is down!')

- jsonrpclibはPython3に対応していないので、jsonrpclib-pelix( https://github.com/tcalmant/jsonrpclib )を使用する

In [ ]:
!pip install jsonrpclib-pelix

In [ ]:
from jsonrpclib import Server

- 使用するVLAN番号を指定する

In [ ]:
VLAN_ID = 10

- vEOSのeAPIへの接続を作成する

In [ ]:
switch1 = Server("http://apitest:password@" + vEOS1 + "/command-api") 
switch2 = Server("http://apitest:password@" + vEOS2 + "/command-api")

- eAPIから取得できることを確認する

In [ ]:
response = switch1.runCmds( 1, [ "show version" ] ) 
print(response)

In [ ]:
response = switch2.runCmds( 1, [ "show version" ] ) 
print(response)

- 以下の通りvEOS1に設定を投入する
  - e1をトランクインターフェースに
  - e2を指定したVLANのアクセスポートにする
  - 指定したVLANのSVIを作成する

In [ ]:
switch1.runCmds(1, [
    "enable",
    "configure", 
    "%s vlan %d" % ('no', VLAN_ID),
    "vlan %s" % (VLAN_ID),
    "interface vlan %d" % (VLAN_ID),
    "ip address 10.1.%d.1/24" % (VLAN_ID),
    "interface e1",
    "switchport mode trunk",
    "interface e2",
    "switchport access vlan %d" % (VLAN_ID)])

- 以下の通りvEOS2に設定を投入する
  - e1をトランクインターフェースに
  - e2を指定したVLANのアクセスポートにする
  - 指定したVLANのSVIを作成する

In [ ]:
switch2.runCmds(1, [
    "enable",
    "configure", 
    "%s vlan %d" % ('no', VLAN_ID),
    "vlan %s" % (VLAN_ID),
    "interface vlan %d" % (VLAN_ID),
    "ip address 10.1.%d.2/24" % (VLAN_ID),
    "interface e1",
    "switchport mode trunk"
])

- 共通のBGP設定コマンドを投入する

In [ ]:
BGP_CMD = [
    "enable",
    "configure", 
    "ip routing",
    "no router bgp",
    "router bgp %d" % (VLAN_ID),
]
switch1.runCmds(1, BGP_CMD)
switch2.runCmds(1, BGP_CMD)

- vEOS1用のBGP設定コマンドを投入する

In [ ]:
switch1.runCmds(1, [
    "enable",
    "configure", 
    "router bgp %d" % (VLAN_ID),
    "neighbor 10.1.%d.2 remote-as %d" % (VLAN_ID, VLAN_ID),
    "neighbor 10.1.%d.2 route-reflector-client" % (VLAN_ID),
])

- vEOS2用のBGP設定コマンドを投入する

In [ ]:
switch2.runCmds(1, [
    "enable",
    "configure", 
    "router bgp %d" % (VLAN_ID),
    "neighbor 10.1.%d.1 remote-as %d" % (VLAN_ID, VLAN_ID),
])

- BGPのネイバが張れているかを確認する

In [ ]:
response = switch1.runCmds(1, [
    "show ip bgp summary"
])

state = response[0]["vrfs"]["default"]["peers"]["10.1.10.2"]["peerState"] 

from time import sleep
while state == "Active":
    sleep(1)
    response = switch1.runCmds(1, [
        "show ip bgp summary"
    ])
    state = response[0]["vrfs"]["default"]["peers"]["10.1.10.2"]["peerState"] 

print("Peer OK")

In [ ]:
response = switch2.runCmds(1, [
    "show ip bgp summary"
])

state = response[0]["vrfs"]["default"]["peers"]["10.1.10.1"]["peerState"] 

while state == "Active":
    sleep(1)
    response = switch2.runCmds(1, [
        "enable",
        "show ip bgp summary"
    ])
    state = response[1]["vrfs"]["default"]["peers"]["10.1.10.1"]["peerState"] 

print("Peer OK")

- vEOS2で新たにプレフィックスを作成して広報させてみる

In [ ]:
switch2.runCmds(1, [
    "enable",
    "configure",
    "interface Loopback0",
    "ip address 192.0.2.1/24",
    "router bgp %d" % (VLAN_ID),
    "network 192.0.2.0/24",
    "end"
])

response = switch2.runCmds(1, [
    "show ip bgp"
])

while '192.0.2.0/24' not in response[0]["vrfs"]["default"]["bgpRouteEntries"].keys():
    sleep(1)
    response = switch2.runCmds(1, [
        "show ip bgp"
    ])

print("Prefix Generation OK!")

- vEOS1に到達しているか確認する

In [ ]:
response = switch1.runCmds(1, [
    "show ip bgp"
])

errCnt = 0
while '192.0.2.0/24' not in response[0]["vrfs"]["default"]["bgpRouteEntries"].keys():
    errCnt += 1

    if errCnt > 5:
        print("Prefix not received.")
        exit()

    sleep(1)
    response = switch2.runCmds(1, [
        "show ip bgp"
    ])

print("Prefix receive OK!")